# Notebook 02: Pelatihan dan Evaluasi Model

Notebook ini mencakup tiga proses utama dalam pembangunan model:
1.  **Hyperparameter Tuning:** Mencari arsitektur CNN dan konfigurasi pelatihan terbaik menggunakan Keras Tuner dengan strategi Bayesian Optimization.
2.  **Pelatihan Final:** Melatih model dengan konfigurasi terbaik pada gabungan data latih dan validasi.
3.  **Evaluasi:** Mengukur kinerja model final pada data uji yang independen.

In [ ]:
# --- Instalasi Pustaka ---
!pip install keras-tuner --quiet

# --- Hubungkan ke Google Drive ---
from google.colab import drive
drive.mount('/content/drive')

# --- Impor Pustaka Utama ---
import os
import json
import numpy as np
import tensorflow as tf
import keras_tuner as kt
from tensorflow.keras import layers, callbacks
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import seaborn as sns
import sys

# --- Inisialisasi ---
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

## Langkah 1: Konfigurasi dan Pemuatan Data

Pada langkah ini, kita akan mendefinisikan parameter-parameter utama untuk proses pelatihan dan memuat dataset `.npy` yang telah disiapkan sebelumnya.

In [ ]:
# --- KONFIGURASI PELATIHAN ---
# Pengguna akan diminta memasukkan path yang relevan.
DATASET_DIR = input("Masukkan path ke direktori dataset (berisi file .npy): ")
OUTPUT_DIR = input("Masukkan path ke direktori output (untuk menyimpan model dan hasil): ")

# Parameter untuk proses tuning dan pelatihan
MAX_TRIALS = 50
EXECUTIONS_PER_TRIAL = 1
TUNING_EPOCHS = 50
FINAL_EPOCHS = 100
PATIENCE = 10
BATCH_SIZE = 64

# --- FUNGSI PEMUATAN DATA ---
def load_data(preprocess_dir):
    print("\nMemuat dataset...")
    try:
        X_train = np.load(os.path.join(preprocess_dir, "X_train.npy"))
        y_train = np.load(os.path.join(preprocess_dir, "y_train.npy"))
        X_val = np.load(os.path.join(preprocess_dir, "X_val.npy"))
        y_val = np.load(os.path.join(preprocess_dir, "y_val.npy"))
        X_test = np.load(os.path.join(preprocess_dir, "X_test.npy"))
        y_test = np.load(os.path.join(preprocess_dir, "y_test.npy"))

        print("\n📊 Data Shapes:")
        print(f"Train: {X_train.shape}, {y_train.shape}")
        print(f"Val:   {X_val.shape}, {y_val.shape}")
        print(f"Test:  {X_test.shape}, {y_test.shape}\n")
        return (X_train, y_train), (X_val, y_val), (X_test, y_test)
    except FileNotFoundError as e:
        print(f"❌ Error: File dataset tidak ditemukan. Pastikan path '{preprocess_dir}' benar.")
        sys.exit(1)

# --- EKSEKUSI PEMUATAN DATA ---
(X_train, y_train), (X_val, y_val), (X_test, y_test) = load_data(DATASET_DIR)

## Langkah 2: Perancangan Arsitektur dan Hyperparameter Tuning

Mendefinisikan ruang pencarian arsitektur CNN dan menjalankan proses tuning.

In [ ]:
# --- FUNGSI UNTUK MEMBANGUN MODEL (UNTUK TUNER) ---
def build_model(hp):
    model = tf.keras.Sequential()
    model.add(layers.Input(shape=(256, 256, 1)))
    num_blocks = hp.Int('num_blocks', 2, 4)
    for i in range(num_blocks):
        kernel_size = hp.Choice(f'kernel_{i}', [3, 5, 7])
        filters = hp.Choice(f'filters_{i}', [32, 64, 128])
        model.add(layers.Conv2D(filters=filters, kernel_size=kernel_size, activation='relu', padding='same'))
        pool_size = hp.Choice(f'pool_{i}', [2, 3, 4])
        model.add(layers.MaxPooling2D(pool_size))
        model.add(layers.BatchNormalization())
        dropout_rate = hp.Float(f'dropout_{i}', 0.2, 0.5, step=0.1)
        model.add(layers.Dropout(dropout_rate))
    model.add(layers.GlobalAveragePooling2D())
    dense_units = hp.Int('dense_units', 64, 512, step=64)
    model.add(layers.Dense(dense_units, activation='relu'))
    model.add(layers.Dropout(hp.Float('dense_dropout', 0.3, 0.7, step=0.1)))
    model.add(layers.Dense(1, activation='sigmoid'))
    optimizer_name = hp.Choice('optimizer', ['adam', 'rmsprop'])
    lr = hp.Float('lr', 1e-4, 1e-2, sampling='log')
    optimizer = tf.keras.optimizers.get(optimizer_name)
    optimizer.learning_rate = lr
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
    )
    return model

# --- EKSEKUSI HYPERPARAMETER TUNING ---
print("\n" + "="*50)
print("MEMULAI HYPERPARAMETER TUNING")
print("="*50)

overwrite_tuning = input("Mulai tuning baru (hapus hasil lama)? (true/false): ").lower() == 'true'

tuner = kt.BayesianOptimization(
    build_model,
    objective='val_auc',
    max_trials=MAX_TRIALS,
    executions_per_trial=EXECUTIONS_PER_TRIAL,
    directory=OUTPUT_DIR,
    project_name='webshell_tuning',
    overwrite=overwrite_tuning
)

tuner.search(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=TUNING_EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[callbacks.EarlyStopping(monitor='val_loss', patience=PATIENCE, restore_best_weights=True)],
    verbose=1
)

print("\n💾 Menyimpan hasil tuning...")
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_hp_path = os.path.join(OUTPUT_DIR, 'best_hyperparameters.json')
with open(best_hp_path, 'w') as f:
    json.dump(best_hps.values, f, indent=2)
print(f"✅ Hyperparameter terbaik disimpan di: {best_hp_path}")

## Langkah 3: Pelatihan Model Final

Membangun model dengan arsitektur terbaik yang ditemukan dan melatihnya pada gabungan data latih dan validasi.

In [ ]:
# --- EKSEKUSI PELATIHAN MODEL FINAL ---
print("\n" + "="*50)
print("MEMULAI PELATIHAN MODEL FINAL")
print("="*50)

final_model = tuner.hypermodel.build(best_hps)
print("Arsitektur Model Final:")
final_model.summary()

X_full = np.concatenate((X_train, X_val))
y_full = np.concatenate((y_train, y_val))

output_model_path = os.path.join(OUTPUT_DIR, 'best_model.keras')
final_callbacks = [
    callbacks.EarlyStopping(monitor='loss', patience=PATIENCE, restore_best_weights=True),
    callbacks.ModelCheckpoint(output_model_path, monitor='loss', save_best_only=True, mode='min'),
    callbacks.CSVLogger(os.path.join(OUTPUT_DIR, 'final_training_log.csv')),
    callbacks.ReduceLROnPlateau(monitor='loss', factor=0.2, patience=5, min_lr=1e-6)
]

print(f"\n🚀 Memulai training... Model akan disimpan di '{output_model_path}'")
history = final_model.fit(
    X_full, y_full,
    epochs=FINAL_EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=final_callbacks,
    verbose=1
)

## Langkah 4: Evaluasi dan Analisis Hasil

Mengevaluasi kinerja model final pada data uji dan memvisualisasikan hasilnya.

In [ ]:
# --- EKSEKUSI EVALUASI ---
print("\n" + "="*50)
print("MEMULAI EVALUASI MODEL")
print("="*50)

y_pred_probs = final_model.predict(X_test, batch_size=BATCH_SIZE)
y_pred_classes = (y_pred_probs > 0.5).astype("int32").flatten()

print("\n📝 Classification Report:")
print(classification_report(y_test, y_pred_classes, target_names=['benign', 'malicious']))

# --- Visualisasi Hasil ---
# Confusion Matrix
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, y_pred_classes)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['benign', 'malicious'], yticklabels=['benign', 'malicious'])
plt.title('Confusion Matrix')
plt.ylabel('Actual Label')
plt.xlabel('Predicted Label')
plt.savefig(os.path.join(OUTPUT_DIR, 'confusion_matrix.png'))
plt.show()

# ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_pred_probs)
roc_auc = auc(fpr, tpr)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.savefig(os.path.join(OUTPUT_DIR, 'roc_curve.png'))
plt.show()

print("\n✅ Semua proses selesai!")